---

## 📦 Passo 1: Importar Bibliotecas

Bibliotecas necessárias para RAG avançado com todas as funcionalidades.

In [1]:
# Verifica instalação dos pacotes principais
import sys

required_packages = {
    'langchain_community': 'langchain-community',
    'langchain_core': 'langchain-core',
    'langchain_ollama': 'langchain-ollama',
    'faiss': 'faiss-cpu ou faiss-gpu',
    'pypdf': 'pypdf',
    'pandas': 'pandas',
    'requests': 'requests',
}

print("=" * 80)
print("🔍 VERIFICAÇÃO DE DEPENDÊNCIAS")
print("=" * 80)

all_installed = True

for module_name, package_name in required_packages.items():
    try:
        __import__(module_name)
        print(f"✅ {package_name}")
    except ImportError:
        print(f"❌ {package_name} - NÃO INSTALADO!")
        all_installed = False

print("=" * 80)

if all_installed:
    print("\n🎉 Todas as dependências estão instaladas!")
else:
    print("\n⚠️ Alguns pacotes estão faltando. Execute:")
    print("   pip install langchain-community langchain-core langchain-ollama faiss-cpu pypdf pandas requests")
    
print("\n")

🔍 VERIFICAÇÃO DE DEPENDÊNCIAS
✅ langchain-community
✅ langchain-core
✅ langchain-ollama
✅ faiss-cpu ou faiss-gpu
✅ pypdf
✅ pandas
✅ requests

🎉 Todas as dependências estão instaladas!




---

## 📚 Passo 2: Importar Bibliotecas

Agora vamos importar todas as bibliotecas necessárias para RAG avançado.

**💡 Nota sobre LCEL:** Este notebook usa **LangChain Expression Language (LCEL)**, a abordagem moderna e recomendada do LangChain. LCEL é mais estável e não depende de classes antigas como `ConversationalRetrievalChain` ou `ConversationBufferMemory` que estão sendo depreciadas.

In [2]:
import os
import hashlib
import json
import requests
import pandas as pd
from datetime import datetime
from pathlib import Path
from typing import List, Dict, Any, Optional

from IPython.display import Markdown, display

# LangChain Core
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings, OllamaLLM
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
from langchain_core.runnables import RunnablePassthrough, RunnableWithMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

print("✅ Bibliotecas importadas com sucesso!")

✅ Bibliotecas importadas com sucesso!


---

## ⚙️ Passo 3: Configurações do Sistema

Definimos todas as configurações do sistema RAG avançado.

### 📊 Configurações Baseadas na Indústria (2024-2025)

Seguindo a tabela de **Tamanhos Padrão da Indústria** vista anteriormente:
- **chunk_size=1800**: Baseline 2024-2025 (sweet spot)
- **chunk_overlap=300**: ~17% do chunk_size
- **k=4**: Chunks recuperados (padrão da indústria)

In [3]:
# Configurações de ambiente
OLLAMA_BASE_URL = 'http://localhost:11434'
BASE_DIR = Path(__file__).parent if "__file__" in globals() else Path.cwd()
PDF_DIR = BASE_DIR.parent.parent / "data" / "pdfs"

# Modelos
EMBEDDING_MODEL = 'embeddinggemma'
LLM_MODEL = 'llama3.2:1b'

# Parâmetros de Chunking (Baseline Indústria 2024-2025)
CHUNK_SIZE = 1800        # Sweet spot recomendado
CHUNK_OVERLAP = 300      # ~17% do chunk_size
SEPARATORS = ["\n\n", "\n", " ", ""]

# Parâmetros de Retrieval
TOP_K_RETRIEVAL = 4      # Chunks iniciais recuperados
TOP_N_RERANK = 3         # Chunks finais após reranking

print("=" * 80)
print("🎯 CONFIGURAÇÃO DO SISTEMA RAG AVANÇADO")
print("=" * 80)
print(f"\n📁 Diretório de PDFs: {PDF_DIR}")
print(f"🤖 Ollama URL: {OLLAMA_BASE_URL}")
print(f"\n🔧 PARÂMETROS:")
print(f"   Chunk Size: {CHUNK_SIZE} chars (Baseline 2024-2025)")
print(f"   Chunk Overlap: {CHUNK_OVERLAP} chars ({CHUNK_OVERLAP/CHUNK_SIZE*100:.0f}%)")
print(f"   Top-K Retrieval: {TOP_K_RETRIEVAL}")
print(f"   Top-N Rerank: {TOP_N_RERANK}")
print(f"\n🧠 MODELOS:")
print(f"   Embedding: {EMBEDDING_MODEL}")
print(f"   LLM: {LLM_MODEL}")
print("=" * 80 + "\n")

🎯 CONFIGURAÇÃO DO SISTEMA RAG AVANÇADO

📁 Diretório de PDFs: e:\01-projetos\11-work\11.34-engenharia-vetorial\data\pdfs
🤖 Ollama URL: http://localhost:11434

🔧 PARÂMETROS:
   Chunk Size: 1800 chars (Baseline 2024-2025)
   Chunk Overlap: 300 chars (17%)
   Top-K Retrieval: 4
   Top-N Rerank: 3

🧠 MODELOS:
   Embedding: embeddinggemma
   LLM: llama3.2:1b



---

## 🔍 Passo 3: Funções Utilitárias

### 3.1 Geração de IDs Únicos (Hash vs Manual)

**Por que usar hashing?**
- ✅ **Determinístico:** Mesmo chunk = mesmo hash
- ✅ **Deduplicação automática:** Detecta chunks duplicados
- ✅ **Rastreável:** ID baseado no conteúdo, não aleatório

**Comparação:**

| Método | Vantagens | Desvantagens | Uso Recomendado |
|--------|-----------|--------------|-----------------|
| **Hash SHA-256** | Deduplicação automática, determinístico | IDs longos (64 chars) | **Produção** (recomendado) |
| **UUID/Manual** | IDs curtos, controle total | Sem deduplicação, pode duplicar | Prototipagem |
| **Incremental** | Simples, legível | Não rastreável, sem deduplicação | Demos/testes |

**Decisão da Indústria (2024):** Hash SHA-256 é o padrão para produção.

In [4]:
def generate_chunk_id(content: str, metadata: Dict[str, Any]) -> str:
    """
    Gera um ID único para o chunk usando SHA-256.
    
    O ID é gerado a partir de:
    - Conteúdo do chunk
    - Metadados relevantes (source, page)
    
    Isso garante:
    1. Mesmo chunk = mesmo ID (deduplicação)
    2. IDs determinísticos (reproduzíveis)
    3. Rastreabilidade (ID baseado em conteúdo)
    
    Args:
        content: Texto do chunk
        metadata: Dicionário de metadados
    
    Returns:
        Hash SHA-256 de 64 caracteres
    """
    # Combina conteúdo + metadados relevantes
    source = metadata.get('source', '')
    page = metadata.get('page', '')
    
    # Cria string única
    unique_string = f"{content}|{source}|{page}"
    
    # Gera hash SHA-256
    hash_object = hashlib.sha256(unique_string.encode('utf-8'))
    chunk_id = hash_object.hexdigest()
    
    return chunk_id


def generate_content_hash(content: str) -> str:
    """
    Gera um hash baseado APENAS no conteúdo (sem metadados).
    
    Use esta função para deduplicação cross-page/cross-document.
    Detecta chunks idênticos independente de fonte ou página.
    
    ⚠️ CUIDADO: Pode remover duplicatas legítimas (ex: disclaimers repetidos)
    
    Args:
        content: Texto do chunk
    
    Returns:
        Hash SHA-256 de 64 caracteres
    """
    # Hash apenas do conteúdo normalizado
    normalized_content = content.strip()  # Remove espaços extras
    hash_object = hashlib.sha256(normalized_content.encode('utf-8'))
    content_hash = hash_object.hexdigest()
    
    return content_hash


def generate_manual_id(index: int, source: str) -> str:
    """
    Gera um ID manual/incremental (alternativa simples).
    
    ⚠️ Não recomendado para produção (sem deduplicação).
    
    Args:
        index: Índice do chunk
        source: Nome do arquivo fonte
    
    Returns:
        ID no formato 'source_chunk_XXX'
    """
    # Extrai nome do arquivo sem extensão
    source_name = Path(source).stem
    
    # Cria ID incremental
    manual_id = f"{source_name}_chunk_{index:04d}"
    
    return manual_id



In [5]:

# Exemplo de uso
exemplo_content = "Este é um chunk de exemplo sobre RAG."
exemplo_metadata = {"source": "manual.pdf", "page": 1}

hash_id = generate_chunk_id(exemplo_content, exemplo_metadata)
content_hash = generate_content_hash(exemplo_content)
manual_id = generate_manual_id(0, "manual.pdf")

print("🔍 COMPARAÇÃO DE MÉTODOS DE ID\n")
print("=" * 80)
print(f"Conteúdo: {exemplo_content}")
print(f"\n1️⃣ Hash SHA-256 (conteúdo + metadados):  {hash_id}")
print(f"   Tamanho: {len(hash_id)} chars")
print(f"   Deduplicação: ✅ Dentro da mesma página/fonte")
print(f"   Detecta cross-page: ❌ Não (páginas diferentes = hash diferente)")

print(f"\n2️⃣ Hash de Conteúdo (apenas texto):  {content_hash}")
print(f"   Tamanho: {len(content_hash)} chars")
print(f"   Deduplicação: ✅ Cross-page e cross-document")
print(f"   Detecta cross-page: ✅ Sim (ignora metadados)")

print(f"\n3️⃣ Manual:        {manual_id}")
print(f"   Tamanho: {len(manual_id)} chars")
print(f"   Determinístico: ❌ Não (depende da ordem)")
print(f"   Deduplicação: ❌ Sem proteção")

print("\n💡 ESCOLHA:")
print("   • Hash completo (opção 1): Permite duplicatas em páginas diferentes")
print("   • Hash de conteúdo (opção 2): Remove TODAS as duplicatas de texto")
print("=" * 80 + "\n")

print("\n💡 RECOMENDAÇÃO: Use Hash SHA-256 em produção!")
print("=" * 80 + "\n")

🔍 COMPARAÇÃO DE MÉTODOS DE ID

Conteúdo: Este é um chunk de exemplo sobre RAG.

1️⃣ Hash SHA-256 (conteúdo + metadados):  830f5cdf91023dc1a39dbe373561c93bf49f61f2ed5e2286459f52e2e894eb13
   Tamanho: 64 chars
   Deduplicação: ✅ Dentro da mesma página/fonte
   Detecta cross-page: ❌ Não (páginas diferentes = hash diferente)

2️⃣ Hash de Conteúdo (apenas texto):  77f6a84dd3b6bbfb355cfccba8485f8dd90b38e5e960c5618e502a2a95bbe23d
   Tamanho: 64 chars
   Deduplicação: ✅ Cross-page e cross-document
   Detecta cross-page: ✅ Sim (ignora metadados)

3️⃣ Manual:        manual_chunk_0000
   Tamanho: 17 chars
   Determinístico: ❌ Não (depende da ordem)
   Deduplicação: ❌ Sem proteção

💡 ESCOLHA:
   • Hash completo (opção 1): Permite duplicatas em páginas diferentes
   • Hash de conteúdo (opção 2): Remove TODAS as duplicatas de texto


💡 RECOMENDAÇÃO: Use Hash SHA-256 em produção!



### 3.2 Extração de Metadados Avançados

**Metadados essenciais para RAG de produção:**

1. **Identificação**
   - `source`: Nome do arquivo
   - `chunk_id`: ID único (hash)
   - `page`: Número da página (para PDFs)

2. **Temporal**
   - `year`: Ano do documento/publicação
   - `ingestion_date`: Data de processamento

3. **Classificação**
   - `doc_type`: Tipo (manual, artigo, relatório, etc.)
   - `category`: Categoria temática
   - `language`: Idioma

4. **Contexto**
   - `chunk_index`: Posição no documento
   - `total_chunks`: Total de chunks do documento
   - `author`: Autor (se disponível)

**Por que isso importa?**
- 🎯 **Filtragem precisa:** "Busque apenas manuais técnicos de 2024"
- 📊 **Analytics:** Rastreie quais fontes são mais citadas
- 🔍 **Debugging:** Identifique problemas por fonte/categoria

In [6]:
def extract_metadata_from_path(file_path: str) -> Dict[str, Any]:
    """
    Extrai metadados do caminho e nome do arquivo.
    
    Convenção de nomenclatura (opcional):
    - nome_do_arquivo_YYYY.pdf  (ano no final)
    - tipo_categoria_nome.pdf
    
    Args:
        file_path: Caminho completo do arquivo
    
    Returns:
        Dicionário com metadados extraídos
    """
    path = Path(file_path)
    filename = path.stem  # Nome sem extensão
    
    # Tenta extrair ano do nome do arquivo (formato: *_YYYY.pdf)
    year = None
    parts = filename.split('_')
    for part in parts:
        if part.isdigit() and len(part) == 4 and part.startswith('20'):
            year = int(part)
            break
    
    # Tenta classificar tipo do documento baseado em palavras-chave
    doc_type = "documento"  # padrão
    filename_lower = filename.lower()
    
    if any(word in filename_lower for word in ['manual', 'guia', 'tutorial']):
        doc_type = "manual"
    elif any(word in filename_lower for word in ['relatorio', 'report']):
        doc_type = "relatorio"
    elif any(word in filename_lower for word in ['artigo', 'paper', 'article']):
        doc_type = "artigo"
    elif any(word in filename_lower for word in ['receita', 'recipe']):
        doc_type = "receita"
    
    # Detecta idioma (simplificado - poderia usar library como langdetect)
    language = "pt-BR"  # padrão
    
    return {
        "source": path.name,
        "source_path": str(path),
        "filename": filename,
        "doc_type": doc_type,
        "year": year,
        "language": language,
        "ingestion_date": datetime.now().isoformat(),
    }


def enrich_chunk_metadata(
    chunk: Document,
    chunk_index: int,
    total_chunks: int,
    custom_metadata: Optional[Dict[str, Any]] = None
) -> Document:
    """
    Enriquece um chunk com metadados adicionais.
    
    Args:
        chunk: Documento/chunk original
        chunk_index: Índice do chunk (0-based)
        total_chunks: Total de chunks do documento
        custom_metadata: Metadados customizados adicionais
    
    Returns:
        Chunk enriquecido com metadados completos
    """
    # Copia metadados existentes
    enriched_metadata = dict(chunk.metadata)
    
    # Adiciona índice e contexto
    enriched_metadata['chunk_index'] = chunk_index
    enriched_metadata['total_chunks'] = total_chunks
    
    # Gera ID único usando hash
    chunk_id = generate_chunk_id(chunk.page_content, enriched_metadata)
    enriched_metadata['chunk_id'] = chunk_id
    
    # Adiciona metadados customizados (se fornecidos)
    if custom_metadata:
        enriched_metadata.update(custom_metadata)
    
    # Cria novo documento com metadados enriquecidos
    enriched_chunk = Document(
        page_content=chunk.page_content,
        metadata=enriched_metadata
    )
    
    return enriched_chunk


In [7]:
# Exemplo de uso
exemplo_path = "manual_tecnico_futebol_2024.pdf"
metadados_extraidos = extract_metadata_from_path(exemplo_path)

print("📋 EXEMPLO DE EXTRAÇÃO DE METADADOS\n")
print("=" * 80)
print(f"Arquivo: {exemplo_path}\n")
print("Metadados extraídos:")
print(json.dumps(metadados_extraidos, indent=2, ensure_ascii=False))
print("=" * 80 + "\n")

📋 EXEMPLO DE EXTRAÇÃO DE METADADOS

Arquivo: manual_tecnico_futebol_2024.pdf

Metadados extraídos:
{
  "source": "manual_tecnico_futebol_2024.pdf",
  "source_path": "manual_tecnico_futebol_2024.pdf",
  "filename": "manual_tecnico_futebol_2024",
  "doc_type": "manual",
  "year": 2024,
  "language": "pt-BR",
  "ingestion_date": "2025-12-18T10:55:51.013534"
}



---

## 📥 Passo 4: Carregamento e Processamento de Documentos

Agora vamos carregar os PDFs e aplicar todas as técnicas de metadados e deduplicação.

### Processo:
1. ✅ Listar todos os PDFs
2. ✅ Extrair metadados do nome do arquivo
3. ✅ Carregar documentos com PyPDFLoader
4. ✅ Enriquecer cada documento com metadados

In [8]:
# Verifica se há PDFs na pasta
pdf_paths = list(PDF_DIR.glob("*.pdf"))

if not pdf_paths:
    print("⚠️ AVISO: Nenhum PDF encontrado!")
    print(f"   Pasta verificada: {PDF_DIR}")
    print("\n💡 DICA: Adicione alguns PDFs na pasta data/pdfs/")
else:
    print(f"📚 Encontrados {len(pdf_paths)} PDFs\n")
    print("=" * 80)
    
    # Carrega e enriquece documentos
    all_documents = []
    
    for pdf_path in pdf_paths:
        # 1. Extrai metadados do arquivo
        file_metadata = extract_metadata_from_path(str(pdf_path))
        
        # 2. Carrega o PDF
        loader = PyPDFLoader(str(pdf_path))
        docs = loader.load()
        
        # 3. Enriquece cada página com metadados
        for doc in docs:
            # Combina metadados do arquivo + metadados da página
            doc.metadata.update(file_metadata)
            all_documents.append(doc)
        
        print(f"✓ {pdf_path.name}")
        print(f"  └── {len(docs)} páginas | Tipo: {file_metadata['doc_type']} | Ano: {file_metadata.get('year', 'N/A')}")
    
    print("=" * 80)
    print(f"\n📊 RESUMO:")
    print(f"   Total de páginas carregadas: {len(all_documents)}")
    
    # Estatísticas por tipo
    tipos = {}
    for doc in all_documents:
        doc_type = doc.metadata.get('doc_type', 'desconhecido')
        tipos[doc_type] = tipos.get(doc_type, 0) + 1
    
    print(f"\n📈 Distribuição por tipo:")
    for tipo, count in tipos.items():
        print(f"   {tipo}: {count} páginas")
    
    print("\n✅ Documentos carregados com metadados enriquecidos!")

incorrect startxref pointer(1)
parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams


📚 Encontrados 7 PDFs

✓ api_documentation_2023.pdf
  └── 3 páginas | Tipo: documento | Ano: 2023
✓ livro_receitas_2025.pdf
  └── 6 páginas | Tipo: receita | Ano: 2025
✓ manual_futebol_2023_copia.pdf
  └── 4 páginas | Tipo: manual | Ano: 2023
✓ manual_futebol_2025.pdf
  └── 4 páginas | Tipo: manual | Ano: 2025
✓ manual_futebol_2025_com_dup.pdf

incorrect startxref pointer(1)
parsing for Object Streams



  └── 5 páginas | Tipo: manual | Ano: 2025
✓ manual_iphone_2025.pdf
  └── 3 páginas | Tipo: manual | Ano: 2025
✓ relatorio_supercopa_2023.pdf
  └── 3 páginas | Tipo: relatorio | Ano: 2023

📊 RESUMO:
   Total de páginas carregadas: 28

📈 Distribuição por tipo:
   documento: 3 páginas
   receita: 6 páginas
   manual: 16 páginas
   relatorio: 3 páginas

✅ Documentos carregados com metadados enriquecidos!


---

## ✂️ Passo 5: Chunking com Deduplicação

Agora vamos dividir os documentos em chunks e aplicar deduplicação usando hash IDs.

### Processo de Deduplicação Dupla:

```python
# Pseudocódigo - Deduplicação Cross-Page
seen_chunk_ids = set()      # Hash completo (conteúdo + página)
seen_content_hashes = set() # Hash apenas de conteúdo

for chunk in all_chunks:
    chunk_id = hash(chunk.content + metadata)  # Inclui página
    content_hash = hash(chunk.content)         # Apenas texto
    
    if chunk_id in seen_chunk_ids:
        # Duplicata EXATA (mesma página) - ignora
        pass
    elif content_hash in seen_content_hashes:
        # Duplicata CROSS-PAGE (páginas diferentes) - ignora
        pass
    else:
        # Chunk único!
        seen_chunk_ids.add(chunk_id)
        seen_content_hashes.add(content_hash)
        unique_chunks.append(chunk)
```

**Por que duplicatas acontecem?**
- 📄 Headers/footers repetidos em PDFs
- 🔄 Documentos processados múltiplas vezes
- 📑 Seções idênticas em documentos diferentes
- 🔁 **Texto copiado em páginas diferentes do mesmo PDF** ← Agora detectado!

### ⚙️ Estratégias de Deduplicação:

| Estratégia | Detecta | Não Detecta | Uso |
|------------|---------|-------------|-----|
| **Hash completo** (conteúdo + página) | Duplicatas na mesma página | Duplicatas cross-page | Preservar contexto de página |
| **Hash de conteúdo** (apenas texto) | TODAS as duplicatas | - | Máxima deduplicação |
| **Dupla verificação** (ambos) | Mesma página + cross-page | - | ✅ **Implementado aqui!** |

In [9]:
# Cria o text splitter com configurações da indústria
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    length_function=len,
    separators=SEPARATORS,
)

print("=" * 80)
print("✂️ CHUNKING COM DEDUPLICAÇÃO CROSS-PAGE")
print("=" * 80)
print(f"\nParâmetros: chunk_size={CHUNK_SIZE}, overlap={CHUNK_OVERLAP}")


✂️ CHUNKING COM DEDUPLICAÇÃO CROSS-PAGE

Parâmetros: chunk_size=1800, overlap=300


In [10]:
# Divide os documentos em chunks
raw_chunks = text_splitter.split_documents(documents=all_documents)

print(f"\n📊 Chunks brutos criados: {len(raw_chunks)}")


📊 Chunks brutos criados: 41


In [11]:
# Enriquece chunks com metadados e IDs únicos
enriched_chunks = []        # Lista de chunks enriquecidos
processed_chunk_ids = set()      # Conjunto de hashs completo (conteúdo + metadados)
processed_content_hashes = set() # Conjunto de hashs apenas de conteúdo

duplicates_samepage = 0     # Duplicatas na mesma página (contador)
duplicates_crosspage = 0    # Duplicatas em páginas diferentes (contador)

In [12]:

for i, chunk in enumerate(raw_chunks):
    # Enriquece com índice e total
    enriched_chunk = enrich_chunk_metadata(
        chunk=chunk,
        chunk_index=i,
        total_chunks=len(raw_chunks),
    )
    
    # Hash completo (conteúdo + metadados)
    chunk_id = enriched_chunk.metadata['chunk_id']
    
    # Hash apenas de conteúdo (para detectar cross-page)
    content_hash = generate_content_hash(chunk.page_content)
    
    # Adiciona o content_hash aos metadados
    enriched_chunk.metadata['content_hash'] = content_hash
    
    # Verifica duplicação
    if chunk_id in processed_chunk_ids:
        # Duplicata exata (mesma página)
        duplicates_samepage += 1
        continue
    elif content_hash in processed_content_hashes:
        # Duplicata cross-page (mesmo conteúdo, página diferente)
        duplicates_crosspage += 1
        continue
    else:
        # Chunk único
        processed_chunk_ids.add(chunk_id)
        processed_content_hashes.add(content_hash)
        enriched_chunks.append(enriched_chunk)

total_duplicates = duplicates_samepage + duplicates_crosspage


In [13]:

print(f"\n📉 DEDUPLICAÇÃO:")
print(f"   ❌ Duplicatas mesma página: {duplicates_samepage}")
print(f"   🔁 Duplicatas cross-page: {duplicates_crosspage}")
print(f"   📊 Total de duplicatas: {total_duplicates}")
print(f"   ✅ Chunks únicos: {len(enriched_chunks)}")
print(f"   📈 Taxa de deduplicação: {total_duplicates/len(raw_chunks)*100:.1f}%")

# Visualização com pandas - Resumo de Deduplicação
print("\n📊 RESUMO DE DEDUPLICAÇÃO:\n")
df_dedup = pd.DataFrame([
    {'Categoria': 'Chunks Brutos', 'Quantidade': len(raw_chunks), 'Percentual': '100%'},
    {'Categoria': 'Duplicatas Mesma Página', 'Quantidade': duplicates_samepage, 'Percentual': f'{duplicates_samepage/len(raw_chunks)*100:.1f}%'},
    {'Categoria': 'Duplicatas Cross-Page', 'Quantidade': duplicates_crosspage, 'Percentual': f'{duplicates_crosspage/len(raw_chunks)*100:.1f}%'},
    {'Categoria': 'Total Removido', 'Quantidade': total_duplicates, 'Percentual': f'{total_duplicates/len(raw_chunks)*100:.1f}%'},
    {'Categoria': 'Chunks Únicos (Final)', 'Quantidade': len(enriched_chunks), 'Percentual': f'{len(enriched_chunks)/len(raw_chunks)*100:.1f}%'},
])
display(df_dedup)



📉 DEDUPLICAÇÃO:
   ❌ Duplicatas mesma página: 0
   🔁 Duplicatas cross-page: 8
   📊 Total de duplicatas: 8
   ✅ Chunks únicos: 33
   📈 Taxa de deduplicação: 19.5%

📊 RESUMO DE DEDUPLICAÇÃO:



,Categoria,Quantidade,Percentual
0,Chunks Brutos,41,100%
1,Duplicatas Mesma Página,0,0.0%
2,Duplicatas Cross-Page,8,19.5%
3,Total Removido,8,19.5%
4,Chunks Únicos (Final),33,80.5%


In [14]:

# Estatísticas de tamanho dos chunks
chunk_sizes = [len(c.page_content) for c in enriched_chunks]

In [15]:

print(f"\n📏 Tamanho dos chunks:")
print(f"   Médio: {sum(chunk_sizes)/len(chunk_sizes):.0f} chars")
print(f"   Mínimo: {min(chunk_sizes)} chars")
print(f"   Máximo: {max(chunk_sizes)} chars")

# Estatísticas detalhadas com pandas
print("\n📊 ESTATÍSTICAS DE TAMANHO (CHARS):\n")
df_stats = pd.DataFrame([
    {'Métrica': 'Média', 'Valor': f'{sum(chunk_sizes)/len(chunk_sizes):.0f}'},
    {'Métrica': 'Mediana', 'Valor': f'{sorted(chunk_sizes)[len(chunk_sizes)//2]:.0f}'},
    {'Métrica': 'Mínimo', 'Valor': f'{min(chunk_sizes)}'},
    {'Métrica': 'Máximo', 'Valor': f'{max(chunk_sizes)}'},
    {'Métrica': 'Desvio Padrão', 'Valor': f'{pd.Series(chunk_sizes).std():.0f}'},
])
display(df_stats)



📏 Tamanho dos chunks:
   Médio: 1192 chars
   Mínimo: 262 chars
   Máximo: 1796 chars

📊 ESTATÍSTICAS DE TAMANHO (CHARS):



,Métrica,Valor
0,Média,1192
1,Mediana,1249
2,Mínimo,262
3,Máximo,1796
4,Desvio Padrão,546


In [16]:

if duplicates_crosspage > 0:
    print(f"\n💡 DETECTADO: {duplicates_crosspage} chunk(s) duplicado(s) em páginas diferentes!")
    print("   Esses chunks foram removidos para evitar redundância.")
    print(f"\n📋 EXPLICAÇÃO:")
    print("   Quando você copia/cola texto em páginas diferentes do mesmo PDF,")
    print("   o sistema detecta que o conteúdo é idêntico (ignorando metadados)")
    print("   e remove automaticamente a duplicata, mantendo apenas a primeira ocorrência.")


print("\n✅ Chunking com deduplicação cross-page concluído!")
print("=" * 80 + "\n")


💡 DETECTADO: 8 chunk(s) duplicado(s) em páginas diferentes!
   Esses chunks foram removidos para evitar redundância.

📋 EXPLICAÇÃO:
   Quando você copia/cola texto em páginas diferentes do mesmo PDF,
   o sistema detecta que o conteúdo é idêntico (ignorando metadados)
   e remove automaticamente a duplicata, mantendo apenas a primeira ocorrência.

✅ Chunking com deduplicação cross-page concluído!



### 🔍 Inspeção de Metadados

Vamos inspecionar os metadados de alguns chunks para verificar se tudo está correto.

In [17]:
# Exibe metadados dos primeiros 3 chunks
print("🔍 INSPEÇÃO DE METADADOS DOS CHUNKS\n")
print("=" * 80 + "\n")

# Visualização com pandas
num_chunks_inspecao = min(5, len(enriched_chunks))
df_metadados = pd.DataFrame([
    {
        'Chunk': f"#{i+1}",
        'Fonte': chunk.metadata.get('source', 'N/A'),
        'Pág': chunk.metadata.get('page', 'N/A'),
        'Tipo': chunk.metadata.get('doc_type', 'N/A'),
        'Ano': chunk.metadata.get('year', 'N/A'),
        'Chunk ID': chunk.metadata.get('chunk_id', 'N/A')[:12] + '...',
        'Content Hash': chunk.metadata.get('content_hash', 'N/A')[:12] + '...',
        'Tamanho': len(chunk.page_content),
        'Preview': chunk.page_content[:60].replace('\n', ' ').strip() + '...'
    }
    for i, chunk in enumerate(enriched_chunks[:num_chunks_inspecao])
])

display(df_metadados)

print("\n💡 OBSERVE:")
print("   • Chunk ID: inclui página (detecta duplicatas na mesma página)")
print("   • Content Hash: apenas conteúdo (detecta duplicatas cross-page)")
print("\n" + "=" * 80 + "\n")

🔍 INSPEÇÃO DE METADADOS DOS CHUNKS




,Chunk,Fonte,Pág,Tipo,Ano,Chunk ID,Content Hash,Tamanho,Preview
0,#1,api_documentation_2023.pdf,0,documento,2023,874557395f61...,047a586ab8ee...,1500,Documentação da API E-commerce Platform REST ...
1,#2,api_documentation_2023.pdf,1,documento,2023,09058b868b45...,c93a1ae08083...,262,GET /products/{product_id} Retorna detalhes de...
2,#3,api_documentation_2023.pdf,2,documento,2023,acc92ce21109...,d8d5bfd57dd4...,926,Endpoints de Pedidos POST /orders Cria um novo...
3,#4,livro_receitas_2025.pdf,0,receita,2025,7adf762f73e5...,8920565c209a...,1524,Livro de Receitas Práticas Sabores do Mundo e...
4,#5,livro_receitas_2025.pdf,1,receita,2025,f0b088c0b436...,0837e2d1d910...,1764,"8. Nos últimos 5 minutos, adicione o manjericã..."



💡 OBSERVE:
   • Chunk ID: inclui página (detecta duplicatas na mesma página)
   • Content Hash: apenas conteúdo (detecta duplicatas cross-page)




---

## 🧮 Passo 6: Criar Embeddings e Vectorstore

Criamos os embeddings e armazenamos no banco vetorial FAISS.

In [18]:
# Configuração dos embeddings
embeddings = OllamaEmbeddings(
    model=EMBEDDING_MODEL,
    base_url=OLLAMA_BASE_URL
)

print("=" * 80)
print("🧮 CRIAÇÃO DE EMBEDDINGS E VECTORSTORE")
print("=" * 80)

print(f"\n🤖 Modelo de embeddings: {EMBEDDING_MODEL}")
print(f"⏳ Criando índice FAISS (pode levar alguns minutos...)\n")

# Cria o vectorstore
vectorstore = FAISS.from_documents(
    documents=enriched_chunks,
    embedding=embeddings
)

print(f"\n✅ Vectorstore criado!")
print(f"📊 Total de vetores indexados: {vectorstore.index.ntotal}")
print(f"📐 Dimensões dos embeddings: {vectorstore.index.d}")
print(f"💾 Memória aproximada: {vectorstore.index.ntotal * vectorstore.index.d * 4 / 1024 / 1024:.2f} MB")

print("=" * 80 + "\n")

🧮 CRIAÇÃO DE EMBEDDINGS E VECTORSTORE

🤖 Modelo de embeddings: embeddinggemma
⏳ Criando índice FAISS (pode levar alguns minutos...)


✅ Vectorstore criado!
📊 Total de vetores indexados: 33
📐 Dimensões dos embeddings: 768
💾 Memória aproximada: 0.10 MB



---

## 🔎 Passo 7: Busca Filtrada por Metadados

Uma das funcionalidades mais poderosas do RAG avançado: **busca filtrada**.

### Como funciona?

```python
# Busca SEM filtro
results = vectorstore.similarity_search("query")  # Busca em TODOS os chunks

# Busca COM filtro
results = vectorstore.similarity_search(
    "query",
    filter={"year": 2024, "doc_type": "manual"}  # Apenas manuais de 2024
)
```

### Casos de Uso na Indústria:

1. **Temporal:** "Busque apenas documentos de 2024"
2. **Tipo:** "Busque apenas em manuais técnicos"
3. **Categoria:** "Busque apenas em documentos de RH"
4. **Combinado:** "Busque em relatórios financeiros de 2023"

**Impacto:** Reduz ruído e melhora precisão em até 40%!

In [19]:
# Exemplo 1: Busca SEM filtro (baseline)
query_teste = "Quais são as formações táticas do futebol no jogo com o Real Metrópolis?"

print("=" * 80)
print("🔍 COMPARAÇÃO: BUSCA COM E SEM FILTRO")
print("=" * 80)

print(f"\n📝 Query: {query_teste}\n")


🔍 COMPARAÇÃO: BUSCA COM E SEM FILTRO

📝 Query: Quais são as formações táticas do futebol no jogo com o Real Metrópolis?



In [20]:

# Busca sem filtro
results_sem_filtro = vectorstore.similarity_search(query_teste, k=5)

print(f"1️⃣ BUSCA SEM FILTRO (baseline):\n")
print(f"   Resultados: {len(results_sem_filtro)}\n")

# Visualização com pandas
df_sem_filtro = pd.DataFrame([
    {
        'Rank': i,
        'Fonte': doc.metadata.get('source', 'N/A'),
        'Página': doc.metadata.get('page', 'N/A'),
        'Tipo': doc.metadata.get('doc_type', 'N/A'),
        'Ano': doc.metadata.get('year', 'N/A'),
        'Preview': doc.page_content[:70].replace('\n', ' ').strip() + '...'
    }
    for i, doc in enumerate(results_sem_filtro, 1)
])

display(df_sem_filtro)


1️⃣ BUSCA SEM FILTRO (baseline):

   Resultados: 5



,Rank,Fonte,Página,Tipo,Ano,Preview
0,1,relatorio_supercopa_2023.pdf,0,relatorio,2023,RELATÓRIO TÉCNICO DE ANÁLISE: FINAL DA SUPERCO...
1,2,relatorio_supercopa_2023.pdf,1,relatorio,2023,centro do campo. O Real Metrópolis aproveitou ...
2,3,relatorio_supercopa_2023.pdf,1,relatorio,2023," Vulnerabilidade: Durante o jogo, o Real Metr..."
3,4,relatorio_supercopa_2023.pdf,0,relatorio,2023,central recuando para a linha de meio e os abe...
4,5,manual_futebol_2025_com_dup.pdf,2,manual,2025,Formações Táticas Clássicas 4-4-2 (Formação E...


In [21]:

# Exemplo 2: Busca COM filtro (apenas manuais)
print(f"\n2️⃣ BUSCA COM FILTRO (apenas doc_type='manual'):\n")

# Verifica se há manuais
tem_manuais = any(c.metadata.get('doc_type') == 'manual' for c in enriched_chunks)

if tem_manuais:
    results_com_filtro = vectorstore.similarity_search(
        query_teste,
        k=TOP_K_RETRIEVAL,
        filter={"doc_type": "manual"}
    )
    print(f"   Resultados: {len(results_com_filtro)}\n")
    
    # Visualização com pandas
    df_com_filtro = pd.DataFrame([
        {
            'Rank': i,
            'Fonte': doc.metadata.get('source', 'N/A'),
            'Página': doc.metadata.get('page', 'N/A'),
            'Tipo': doc.metadata.get('doc_type', 'N/A'),
            'Preview': doc.page_content[:70].replace('\n', ' ').strip() + '...'
        }
        for i, doc in enumerate(results_com_filtro, 1)
    ])
    
    display(df_com_filtro)
else:
    print("   ⚠️ Nenhum manual encontrado na base (filtro não aplicável)")



2️⃣ BUSCA COM FILTRO (apenas doc_type='manual'):

   Resultados: 4



,Rank,Fonte,Página,Tipo,Preview
0,1,manual_futebol_2025_com_dup.pdf,2,manual,Formações Táticas Clássicas 4-4-2 (Formação E...
1,2,manual_futebol_2023_copia.pdf,2,manual,Formações Táticas Clássicas 4-4-2 (Formação Eq...
2,3,manual_futebol_2025_com_dup.pdf,3,manual,4-2-3-1 (Formação Moderna de Controle): Siste...
3,4,manual_futebol_2023_copia.pdf,3,manual,4-2-3-1 (Formação Moderna de Controle): Sistem...


In [22]:

# Exemplo 3: Busca por ano (se disponível)
anos_disponiveis = set(c.metadata.get('year') for c in enriched_chunks if c.metadata.get('year'))

if anos_disponiveis:
    ano_exemplo = list(anos_disponiveis)[0]
    print(f"\n3️⃣ BUSCA COM FILTRO (apenas year={ano_exemplo}):")
    
    results_por_ano = vectorstore.similarity_search(
        query_teste,
        k=TOP_K_RETRIEVAL,
        filter={"year": ano_exemplo}
    )
    print(f"   Resultados: {len(results_por_ano)}")
    for i, doc in enumerate(results_por_ano, 1):
        print(f"   {i}. {doc.metadata.get('source', 'N/A')} (ano: {doc.metadata.get('year', 'N/A')})")

print("\n💡 INSIGHT: Filtros reduzem ruído e melhoram precisão!")
print("=" * 80 + "\n")


3️⃣ BUSCA COM FILTRO (apenas year=2025):
   Resultados: 4
   1. manual_futebol_2025_com_dup.pdf (ano: 2025)
   2. manual_futebol_2025_com_dup.pdf (ano: 2025)
   3. manual_futebol_2025_com_dup.pdf (ano: 2025)
   4. manual_futebol_2025_com_dup.pdf (ano: 2025)

💡 INSIGHT: Filtros reduzem ruído e melhoram precisão!



---

## 📊 Passo 8: Reranking (Reordenação de Resultados)

**O que é Reranking?**

Reranking é o processo de **reordenar** os chunks recuperados para colocar os mais relevantes no topo.

### Por que precisamos de Reranking?

A busca vetorial (cosine similarity) é **rápida** mas **imperfeita**:
- Pode ranquear chunks genéricos no topo
- Não captura nuances de relevância contextual
- Ordem pode não refletir importância real

### Estratégias de Reranking:

| Estratégia | Descrição | Uso |
|------------|-----------|-----|
| **Cross-Encoder** | Modelo que avalia query+chunk juntos | Produção (melhor qualidade) |
| **Boosting por Metadados** | Aumenta score de chunks com metadados relevantes | Rápido, eficaz |
| **Diversificação** | Garante variedade de fontes | Multi-documento |
| **Recência** | Prioriza documentos mais recentes | Notícias, compliance |

**Implementação Simplificada:** Vamos usar **boosting por metadados** (rápido e eficaz) com recência.

In [23]:
def rerank_with_metadata_boosting(
    query: str,
    chunks: List[Document],
    boost_rules: Optional[Dict[str, float]] = None
) -> List[tuple[Document, float]]:
    """
    Reordena chunks aplicando boosting baseado em metadados.
    
    Estratégia:
    1. Cada chunk tem um score base (da similaridade vetorial)
    2. Aplicamos multiplicadores (boosts) baseados em metadados
    3. Reordenamos por score final
    
    Args:
        query: Query do usuário
        chunks: Chunks recuperados pela busca vetorial
        boost_rules: Regras de boosting (ex: {"doc_type": {"manual": 1.5}})
    
    Returns:
        Lista de (chunk, score_final) ordenada por relevância
    """
    if boost_rules is None:
        # Regras padrão de boosting
        boost_rules = {
            "doc_type": {
                "manual": 1.3,      # Manuais têm prioridade
                "artigo": 1.2,
                "relatorio": 1.1,
            },
            "recency_boost": True,  # Documentos mais recentes ganham boost
        }
    
    reranked = []
    current_year = datetime.now().year
    
    for i, chunk in enumerate(chunks):
        # Score base 
        base_score = 1.0 / (i + 1)
        
        # Aplica boosting por tipo de documento
        doc_type = chunk.metadata.get('doc_type', 'documento')
        type_boost = boost_rules.get('doc_type', {}).get(doc_type, 1.0)
        
        # Aplica boosting por recência (documentos de 1-2 anos = boost)
        recency_boost = 1.0
        if boost_rules.get('recency_boost', False):
            year = chunk.metadata.get('year')
            if year:
                age = current_year - year
                if age <= 1:
                    recency_boost = 1.3  # Muito recente
                elif age <= 2:
                    recency_boost = 1.15  # Recente
                elif age <= 5:
                    recency_boost = 1.0  # Neutro
                else:
                    recency_boost = 0.9  # Antigo
        
        # Score final
        final_score = base_score * type_boost * recency_boost
        
        reranked.append((chunk, final_score, base_score, type_boost, recency_boost))
    
    # Ordena por score final (decrescente)
    reranked.sort(key=lambda x: x[1], reverse=True)
    
    return reranked



In [24]:

# Teste de reranking
print("=" * 80)
print("📊 RERANKING COM METADATA BOOSTING E RECÊNCIA")
print("=" * 80)

print(f"\n📝 Query: {query_teste}\n")

# Busca inicial
initial_results = vectorstore.similarity_search(query_teste, k=6)

# Se quiser ver os resultados antes do reranking em modo texto:
# print(f"\n2️⃣ ORDEM APÓS RERANKING:")
# for i, (doc, score) in enumerate(reranked_results, 1):
#     print(f"   {i}. {doc.metadata.get('source', 'N/A')} | Tipo: {doc.metadata.get('doc_type', 'N/A')} | Ano: {doc.metadata.get('year', 'N/A')} | Score: {score:.3f} | Conteúdo: {doc.page_content[:50].replace(chr(10), ' ')}...")

# Cria DataFrame para visualização dos resultados originais
print(f"1️⃣ ORDEM ORIGINAL (busca vetorial):\n")
df_original = pd.DataFrame([
    {
        'Posição': i,
        'Fonte': doc.metadata.get('source', 'N/A'),
        'Página': doc.metadata.get('page', 'N/A'),
        'Tipo': doc.metadata.get('doc_type', 'N/A'),
        'Ano': doc.metadata.get('year', 'N/A'),
        'Score Base': f"{1.0/(i+1):.3f}",
        'Preview': doc.page_content[:60].replace('\n', ' ').strip() + '...'
    }
    for i, doc in enumerate(initial_results, 1)
])

display(df_original)


📊 RERANKING COM METADATA BOOSTING E RECÊNCIA

📝 Query: Quais são as formações táticas do futebol no jogo com o Real Metrópolis?

1️⃣ ORDEM ORIGINAL (busca vetorial):



,Posição,Fonte,Página,Tipo,Ano,Score Base,Preview
0,1,relatorio_supercopa_2023.pdf,0,relatorio,2023,0.500,RELATÓRIO TÉCNICO DE ANÁLISE: FINAL DA SUPERCO...
1,2,relatorio_supercopa_2023.pdf,1,relatorio,2023,0.333,centro do campo. O Real Metrópolis aproveitou ...
2,3,relatorio_supercopa_2023.pdf,1,relatorio,2023,0.250," Vulnerabilidade: Durante o jogo, o Real Metr..."
3,4,relatorio_supercopa_2023.pdf,0,relatorio,2023,0.200,central recuando para a linha de meio e os abe...
4,5,manual_futebol_2025_com_dup.pdf,2,manual,2025,0.167,Formações Táticas Clássicas 4-4-2 (Formação E...
5,6,manual_futebol_2023_copia.pdf,2,manual,2023,0.143,Formações Táticas Clássicas 4-4-2 (Formação Eq...


In [25]:
# Aplica reranking
reranked_results = rerank_with_metadata_boosting(query_teste, initial_results)

# Cria DataFrame para visualização dos resultados reranqueados
print(f"\n2️⃣ ORDEM APÓS RERANKING:\n")
df_reranked = pd.DataFrame([
    {
        'Posição': i,
        'Fonte': doc.metadata.get('source', 'N/A'),
        'Página': doc.metadata.get('page', 'N/A'),
        'Tipo': doc.metadata.get('doc_type', 'N/A'),
        'Ano': doc.metadata.get('year', 'N/A'),
        'Base Score': f"{base_score:.3f}",
        'Type Boost': f"{type_boost:.3f}",
        'Recency Boost': f"{recency_boost:.3f}",
        'Score': f"{score:.3f}",
        'Preview': doc.page_content[:60].replace('\n', ' ').strip() + '...'
    }
    for i, (doc, score, base_score, type_boost, recency_boost) in enumerate(reranked_results, 1)
])

display(df_reranked)



2️⃣ ORDEM APÓS RERANKING:



,Posição,Fonte,Página,Tipo,Ano,Base Score,Type Boost,Recency Boost,Score,Preview
0,1,relatorio_supercopa_2023.pdf,0,relatorio,2023,1.000,1.100,1.150,1.265,RELATÓRIO TÉCNICO DE ANÁLISE: FINAL DA SUPERCO...
1,2,relatorio_supercopa_2023.pdf,1,relatorio,2023,0.500,1.100,1.150,0.632,centro do campo. O Real Metrópolis aproveitou ...
2,3,relatorio_supercopa_2023.pdf,1,relatorio,2023,0.333,1.100,1.150,0.422," Vulnerabilidade: Durante o jogo, o Real Metr..."
3,4,manual_futebol_2025_com_dup.pdf,2,manual,2025,0.200,1.300,1.300,0.338,Formações Táticas Clássicas 4-4-2 (Formação E...
4,5,relatorio_supercopa_2023.pdf,0,relatorio,2023,0.250,1.100,1.150,0.316,central recuando para a linha de meio e os abe...
5,6,manual_futebol_2023_copia.pdf,2,manual,2023,0.167,1.300,1.150,0.249,Formações Táticas Clássicas 4-4-2 (Formação Eq...


In [31]:
print(f"\n📈 ANÁLISE DE RERANKING:")

# Analisa as mudanças entre ordem original e reranqueada
mudancas_detalhes = []
for i in range(len(initial_results)):
    doc_orig = initial_results[i]
    doc_rerank, score_final, base_score, type_boost, recency_boost = reranked_results[i]
    
    # Encontra a posição original do documento reranqueado
    pos_original = None
    for j, doc in enumerate(initial_results):
        if doc.metadata.get("chunk_id") == doc_rerank.metadata.get("chunk_id"):
            pos_original = j
            break
    
    if pos_original != i:
        mudanca = "↑ Promovido" if pos_original > i else "↓ Rebaixado"
        mudancas_detalhes.append({
            'Pos. Original': pos_original + 1,
            'Pos. Reranked': i + 1,
            'Mudança': mudanca,
            'Fonte': doc_rerank.metadata.get('source', 'N/A'),
            'Tipo': doc_rerank.metadata.get('doc_type', 'N/A'),
            'Ano': doc_rerank.metadata.get('year', 'N/A'),
            'Score Final': f"{score_final:.3f}",
            'Type Boost': f"{type_boost:.2f}x",
            'Recency Boost': f"{recency_boost:.2f}x"
        })

# Exibe resultados
if mudancas_detalhes:
    print(f"\n   ✓ {len(mudancas_detalhes)} posições foram alteradas pelo reranking\n")
    df_mudancas = pd.DataFrame(mudancas_detalhes)
    display(df_mudancas)
else:
    print(f"\n   ℹ️ Nenhuma mudança de posição (ordem original mantida)")

# Seleciona top-N após reranking
final_chunks = [doc for doc, *_ in reranked_results[:TOP_N_RERANK]]

print("💡 INSIGHT: Reranking priorizou documentos mais relevantes baseado em metadados!")
print("=" * 80 + "\n")


📈 ANÁLISE DE RERANKING:

   ✓ 2 posições foram alteradas pelo reranking



,Pos. Original,Pos. Reranked,Mudança,Fonte,Tipo,Ano,Score Final,Type Boost,Recency Boost
0,5,4,↑ Promovido,manual_futebol_2025_com_dup.pdf,manual,2025,0.338,1.30x,1.30x
1,4,5,↓ Rebaixado,relatorio_supercopa_2023.pdf,relatorio,2023,0.316,1.10x,1.15x


💡 INSIGHT: Reranking priorizou documentos mais relevantes baseado em metadados!



---

## 💬 Passo 9: Geração de Resposta com Contexto

Agora vamos gerar a resposta usando os chunks reranqueados.

### Melhorias no Prompt:
1. ✅ Instrução clara
2. ✅ Contexto rico (chunks reranqueados)
3. ✅ Citação de fontes
4. ✅ Validação (diga "não sei" se não tiver certeza)

In [32]:
# Template de prompt otimizado
template_rag = """Você é um assistente especializado em responder perguntas baseado em documentos fornecidos.

INSTRUÇÕES:
1. Use APENAS o contexto abaixo para responder
2. Se a resposta não estiver no contexto, diga: "Não encontrei essa informação nos documentos fornecidos."
3. SEMPRE cite as fontes (nome do arquivo e página)
4. Seja objetivo e preciso

CONTEXTO:
{context}

PERGUNTA: {question}

RESPOSTA (cite as fontes):"""

prompt_rag = PromptTemplate(
    template=template_rag,
    input_variables=['context', 'question']
)

# Formata o contexto
formatted_context = '\n\n---\n\n'.join([
    f"[Fonte: {chunk.metadata.get('source', 'N/A')}, Página: {chunk.metadata.get('page', 'N/A')}]\n{chunk.page_content}"
    for chunk in final_chunks
])

# Cria o LLM
llm = OllamaLLM(model=LLM_MODEL, base_url=OLLAMA_BASE_URL)

# Preenche o prompt
prompt_filled = prompt_rag.invoke({
    'context': formatted_context,
    'question': query_teste
})

# Gera resposta
print("=" * 80)
print("💬 GERAÇÃO DE RESPOSTA")
print("=" * 80)
print(f"\n📝 Pergunta: {query_teste}\n")

response = llm.invoke(prompt_filled)

display(Markdown(f"**🤖 Resposta:**\n\n{response}"))

print("\n" + "=" * 80 + "\n")

💬 GERAÇÃO DE RESPOSTA

📝 Pergunta: Quais são as formações táticas do futebol no jogo com o Real Metrópolis?



**🤖 Resposta:**

De acordo com os relatórios, a equipe do Real Metrópolis adotou a formação 4-2-3-1. Essa disposição de defensores em círculo central e dois meias de contenção para proteger a defesa foi uma das características mais destacadas da tática utilizada pelo time no jogo contra o União do Litoral.

---

## 🧠 Passo 10: Chatbot RAG com LangChain

A funcionalidade principal: **RAG (Retrieval-Augmented Generation)**.

### O que é RAG?

RAG combina **busca** + **geração** para criar respostas baseadas em documentos:

```
👤 Usuário: "Quais são as formações do futebol?"
🔍 Sistema: Busca chunks relevantes nos documentos
🤖 LLM: Gera resposta usando os chunks como contexto
```

### LCEL (LangChain Expression Language):

Usamos a abordagem moderna do LangChain, que é:
- ✅ **Modular:** Componentes reutilizáveis
- ✅ **Flexível:** Fácil customização
- ✅ **Estável:** Não depende de classes depreciadas

### Pipeline LCEL:

```python
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)
```

**Fluxo:** Pergunta → Retrieval → Format → Prompt → LLM → Parse

In [ ]:
# Template de prompt RAG
template_conversacional = """Você é um assistente especializado com acesso a documentos.

INSTRUÇÕES:
1. Use o contexto fornecido para responder
2. Se a resposta não estiver no contexto, diga que não sabe
3. Sempre cite as fontes (arquivo e página)
4. Seja objetivo e preciso

CONTEXTO:
{context}

PERGUNTA: {question}

RESPOSTA (cite as fontes):"""

prompt_conversacional = ChatPromptTemplate.from_template(template_conversacional)

# Cria retriever
retriever = vectorstore.as_retriever(
    search_kwargs={"k": TOP_K_RETRIEVAL}
)

# Função para formatar documentos
def format_docs(docs):
    return "\n\n---\n\n".join([
        f"[Fonte: {doc.metadata.get('source', 'N/A')}, Página: {doc.metadata.get('page', 'N/A')}]\n{doc.page_content}"
        for doc in docs
    ])

# Cria a chain RAG usando LCEL
rag_chain = (
    {
        "context": retriever | format_docs, 
        "question": RunnablePassthrough()
    }
    | prompt_conversacional
    | llm
    | StrOutputParser()
)

print("=" * 80)
print("🧠 CHATBOT RAG CONFIGURADO")
print("=" * 80)
print("\n✅ Sistema configurado!")
print("💡 Usando abordagem LCEL moderna do LangChain\n")

🧠 CHATBOT RAG CONFIGURADO

✅ Sistema configurado!
💡 Usando abordagem LCEL moderna do LangChain



### Simulação de Conversa RAG

Vamos testar o sistema com perguntas sobre os documentos.

In [34]:
# Simulação de conversa
conversas = [
    "Quais são as principais formações táticas do futebol?",
    "Qual delas é mais ofensiva?",
    "E a mais defensiva?",
]

print("=" * 80)
print("💬 SIMULAÇÃO DE CONVERSA")
print("=" * 80)

for i, pergunta in enumerate(conversas, 1):
    print(f"\n{'─' * 80}")
    print(f"👤 Turno {i}: {pergunta}")
    print(f"{'─' * 80}")
    
    # Processa a pergunta
    resposta = rag_chain.invoke(pergunta)
    
    # Exibe resposta
    display(Markdown(f"**🤖 Resposta:**\n\n{resposta}"))
    
    # Busca documentos para mostrar fontes
    docs = retriever.invoke(pergunta)
    if docs:
        print("\n📚 Fontes utilizadas:")
        for doc in docs[:2]:  # Top 2 fontes
            print(f"   • {doc.metadata.get('source', 'N/A')} (pág. {doc.metadata.get('page', 'N/A')})")

print("\n" + "=" * 80)
print("\n✅ Conversação concluída!")
print("=" * 80 + "\n")

💬 SIMULAÇÃO DE CONVERSA

────────────────────────────────────────────────────────────────────────────────
👤 Turno 1: Quais são as principais formações táticas do futebol?
────────────────────────────────────────────────────────────────────────────────


**🤖 Resposta:**

As principais formações táticas do futebol incluem:

- 4-4-2 (Formação Equilibrada Clássica)
- 4-3-3 (Formação Ofensiva com Wingers)
- 4-2-3-1 (Formação Moderna de Controle)
- Variações destas, como 4-2-3-1 falso 9 e 4-3-3 falso 9.

Essas formações são frequentemente usadas por equipes para otimizar sua estrutura e controlar o jogo.


📚 Fontes utilizadas:
   • manual_futebol_2023_copia.pdf (pág. 2)
   • manual_futebol_2025_com_dup.pdf (pág. 2)

────────────────────────────────────────────────────────────────────────────────
👤 Turno 2: Qual delas é mais ofensiva?
────────────────────────────────────────────────────────────────────────────────


**🤖 Resposta:**

Considerando a análise dos conceitos táticos e formações apresentados, a variante 4-3-3 (Formação Ofensiva com Wingers) parece ser a mais ofensiva. Isso se deve ao fato de que ela é caracterizada por:

* Uma dupla de extremos/pontas abertos, o que facilita as jogadas aéreas e os cruzamentos;
* A presença de meias laterais velozes nas pontas, o que permite uma rápida mudança de direção e uma boa cobertura defensiva;
* O uso de um volante crucial como "pivô" entre defesa e ataque, o que ajuda a controlar o ritmo do jogo e a manter a posse de bola.

Além disso, a variante 4-3-3 também apresenta características como:

* Alta amplitude ofensiva (três atacantes abertos);
* Domínio de posse com triângulos no meio-campo;
* Pontas cortam para dentro ou ficam abertas para receber.

Em comparação, as outras variantes apresentadas têm características menos ofensivas:

* A variante 4-4-2 é mais defensiva e tem uma cobertura defensiva mais boa, com uma linha de zagueiros com boa liberdade;
* A variante 4-3-3 falso 9 é mais ofensiva, mas apresenta algumas características menos ofensivas do que a variante original.
* A variante 3-5-2 é mais defensiva e tem uma cobertura defensiva mais boa, com três zagueiros e um meio-campo povoado.

Em resumo, a variante 4-3-3 (Formação Ofensiva com Wingers) parece ser a mais ofensiva devido ao seu uso de extremos/pontas abertos, meias laterais velozes nas pontas e o papel do volante como "pivô" entre defesa e ataque.


📚 Fontes utilizadas:
   • manual_futebol_2025_com_dup.pdf (pág. 4)
   • manual_futebol_2023_copia.pdf (pág. 2)

────────────────────────────────────────────────────────────────────────────────
👤 Turno 3: E a mais defensiva?
────────────────────────────────────────────────────────────────────────────────


**🤖 Resposta:**

Com base nas informações fornecidas, a estrutura do sistema com três zagueiros e domínio do meio-campo é considerada mais defensiva. Isso se deve ao fato de que:

* O uso de três zagueiros centrais (3-5-2) aumenta a superioridade numérica no meio-campo, o que pode ser uma característica de um sistema defensivo.
* A presença de dois ala-defensores laterais (zagueiros centrais) também contribui para a defesa, pois eles podem marcar individualmente ou em zona e ter liberdade para marcar no meio-campo.
* A estrutura do meio-campo, com dois meias centrais e duas alas, pode ser uma característica de um sistema defensivo, pois permite uma boa cobertura defensiva e controle do jogo.

No entanto, é importante notar que a resposta final não deve ser interpretada como uma crítica ao futebol em si. A defesa é uma parte importante da jogabilidade e pode ser uma estratégia eficaz para um time.

Fontes:

* Manual Futebol 2025 com Dup (Página: 2)
* Formações Táticas Clássicas 4-4-2 (Formação Equilibrada Clássica) (Página: 1)
* Formações Táticas Clássicas 3-5-2 (Formação com Ala-defensores) (Página: 1)
* Variação 4-3-3 falso 9 (Página: 1)
* Manual Futebol 2023 Copia (Página: 2)
* Formações Táticas Clássicas 4-4-2 (Formação Ofensiva com Wingers) (Página: 1)

Lembre-se de que a defesa é uma parte importante da jogabilidade e pode ser uma estratégia eficaz para um time.


📚 Fontes utilizadas:
   • manual_futebol_2025_com_dup.pdf (pág. 2)
   • manual_futebol_2023_copia.pdf (pág. 2)


✅ Conversação concluída!



### Inspecionar Sistema

Vamos verificar as informações do sistema configurado.

In [35]:
# Inspeciona o sistema
print("=" * 80)
print("🔍 INFORMAÇÕES DO SISTEMA RAG")
print("=" * 80)

print(f"\n📊 Estatísticas da última execução:")
print(f"   Perguntas processadas: {len(conversas)}")
print(f"   Retrieval: Top-{TOP_K_RETRIEVAL} chunks")
print(f"   Modelo LLM: {LLM_MODEL}")
print(f"   Modelo Embedding: {EMBEDDING_MODEL}")

print("\n💡 NOTA: Para implementar memória conversacional completa,")
print("   use RunnableWithMessageHistory do langchain_core.")
print("\n📚 Referência: https://python.langchain.com/docs/expression_language/how_to/message_history")

print("=" * 80)

🔍 INFORMAÇÕES DO SISTEMA RAG

📊 Estatísticas da última execução:
   Perguntas processadas: 3
   Retrieval: Top-4 chunks
   Modelo LLM: llama3.2:1b
   Modelo Embedding: embeddinggemma

💡 NOTA: Para implementar memória conversacional completa,
   use RunnableWithMessageHistory do langchain_core.

📚 Referência: https://python.langchain.com/docs/expression_language/how_to/message_history


---

## 📊 Passo 11: Análise e Estatísticas do Sistema

Vamos analisar o desempenho e características do nosso sistema RAG avançado.

In [36]:
print("=" * 80)
print("📊 ANÁLISE COMPLETA DO SISTEMA RAG AVANÇADO")
print("=" * 80)

# Estatísticas de documentos
print("\n1️⃣ INGESTÃO DE DOCUMENTOS:")
print(f"   PDFs carregados: {len(pdf_paths)}")
print(f"   Páginas totais: {len(all_documents)}")
print(f"   Chunks brutos: {len(raw_chunks)}")
print(f"   Chunks únicos (pós-deduplicação): {len(enriched_chunks)}")
print(f"\n   DEDUPLICAÇÃO DETALHADA:")
print(f"   • Duplicatas mesma página: {duplicates_samepage}")
print(f"   • Duplicatas cross-page: {duplicates_crosspage}")
print(f"   • Total removido: {total_duplicates}")
print(f"   • Taxa de deduplicação: {total_duplicates/len(raw_chunks)*100:.1f}%")


📊 ANÁLISE COMPLETA DO SISTEMA RAG AVANÇADO

1️⃣ INGESTÃO DE DOCUMENTOS:
   PDFs carregados: 7
   Páginas totais: 28
   Chunks brutos: 41
   Chunks únicos (pós-deduplicação): 33

   DEDUPLICAÇÃO DETALHADA:
   • Duplicatas mesma página: 0
   • Duplicatas cross-page: 8
   • Total removido: 8
   • Taxa de deduplicação: 19.5%


In [37]:

# Estatísticas de chunks
print("\n2️⃣ CARACTERÍSTICAS DOS CHUNKS:")
print(f"   Tamanho configurado: {CHUNK_SIZE} chars")
print(f"   Overlap configurado: {CHUNK_OVERLAP} chars ({CHUNK_OVERLAP/CHUNK_SIZE*100:.0f}%)")
print(f"   Tamanho médio real: {sum(chunk_sizes)/len(chunk_sizes):.0f} chars")
print(f"   Expansão de chunking: {len(enriched_chunks)/len(all_documents):.1f}x")



2️⃣ CARACTERÍSTICAS DOS CHUNKS:
   Tamanho configurado: 1800 chars
   Overlap configurado: 300 chars (17%)
   Tamanho médio real: 1192 chars
   Expansão de chunking: 1.2x


In [38]:

# Estatísticas de metadados
tipos_unicos = set(c.metadata.get('doc_type') for c in enriched_chunks)
anos_unicos = set(c.metadata.get('year') for c in enriched_chunks if c.metadata.get('year'))

print("\n3️⃣ METADADOS:")
print(f"   Tipos de documentos: {len(tipos_unicos)} → {list(tipos_unicos)}")
print(f"   Anos cobertos: {len(anos_unicos)} → {sorted(anos_unicos) if anos_unicos else 'N/A'}")



3️⃣ METADADOS:
   Tipos de documentos: 4 → ['relatorio', 'documento', 'manual', 'receita']
   Anos cobertos: 2 → [2023, 2025]


In [39]:

# Estatísticas do vectorstore
print("\n4️⃣ VECTORSTORE:")
print(f"   Vetores indexados: {vectorstore.index.ntotal}")
print(f"   Dimensões: {vectorstore.index.d}")
print(f"   Memória estimada: {vectorstore.index.ntotal * vectorstore.index.d * 4 / 1024 / 1024:.2f} MB")



4️⃣ VECTORSTORE:
   Vetores indexados: 33
   Dimensões: 768
   Memória estimada: 0.10 MB


In [40]:

# Parâmetros de retrieval
print("\n5️⃣ RETRIEVAL & RERANKING:")
print(f"   Top-K inicial: {TOP_K_RETRIEVAL}")
print(f"   Top-N pós-rerank: {TOP_N_RERANK}")
print(f"   Redução: {(1 - TOP_N_RERANK/TOP_K_RETRIEVAL)*100:.0f}%")



5️⃣ RETRIEVAL & RERANKING:
   Top-K inicial: 4
   Top-N pós-rerank: 3
   Redução: 25%


In [41]:

# Chain RAG
print("\n6️⃣ CHAIN RAG:")
print(f"   Tipo: LCEL (LangChain Expression Language)")
print(f"   Componentes: Retriever → Formatter → Prompt → LLM → Parser")

print("\n" + "=" * 80)
print("\n✅ Sistema RAG Avançado totalmente operacional!")
print("=" * 80 + "\n")


6️⃣ CHAIN RAG:
   Tipo: LCEL (LangChain Expression Language)
   Componentes: Retriever → Formatter → Prompt → LLM → Parser


✅ Sistema RAG Avançado totalmente operacional!



---

## 🎓 Resumo

### 🏆 O que você aprendeu neste notebook:

#### 1. **Pipeline RAG Completo da Indústria**
   - ✅ Ingestão → Processamento → Embedding → Retrieval → Reranking → Geração
   - ✅ Fluxo completo de produção (2024-2025)

#### 2. **Prevenção de Duplicação Avançada**
   - ✅ IDs hash SHA-256 (determinísticos e únicos)
   - ✅ **Deduplicação dupla:** mesma página + cross-page
   - ✅ **Detecta duplicatas em páginas diferentes** (conteúdo idêntico)
   - ✅ Hash de conteúdo (ignora metadados) + Hash completo (inclui metadados)
   - ✅ Comparação: Hash vs Manual vs Incremental

#### 3. **Metadados Estruturados**
   - ✅ Extração de metadados (source, tipo, ano, etc.)
   - ✅ Enriquecimento de chunks
   - ✅ Rastreabilidade completa

#### 4. **Busca Filtrada**
   - ✅ Filtros por metadados (tipo, ano, categoria)
   - ✅ Redução de ruído (até 40% mais precisão)
   - ✅ Casos de uso práticos

#### 5. **Reranking**
   - ✅ Metadata boosting (tipo de doc, recência)
   - ✅ Reordenação inteligente
   - ✅ Seleção de top-N chunks

#### 6. **RAG Chain com LCEL**
   - ✅ LangChain Expression Language (abordagem moderna)
   - ✅ Pipeline modular e flexível
   - ✅ Geração baseada em contexto recuperado

---



## 💪 Exercícios Práticos:

### 🎯 Exercício 1: Teste Diferentes Filtros
Experimente criar buscas com filtros customizados:
- Busque apenas documentos de um ano específico
- Combine filtros (tipo + ano)
- Compare precisão com e sem filtros


In [42]:
# Código aqui


### 🎯 Exercício 2: Ajuste o Reranking
Modifique as regras de boosting:
- Dê mais peso a documentos recentes
- Priorize um tipo de documento específico
- Crie regras customizadas para seu domínio


In [43]:
# Código aqui



### 🎯 Exercício 3: Implemente Memória Conversacional
Adicione memória para contexto multi-turn:
- Use RunnableWithMessageHistory do langchain_core
- Implemente histórico persistente com InMemoryChatMessageHistory
- Teste conversas com contexto acumulado
- Referência: https://python.langchain.com/docs/expression_language/how_to/message_history


In [44]:
# Código aqui



---

**Parabéns! 🎉** 

Você agora domina técnicas avançadas de RAG usadas em **sistemas de produção** na indústria!

Continue experimentando e adaptando para seu caso de uso específico. 🚀